**Importing necessary libraries and dataset**
**bold text**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


# Loading and preprocessing the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Expand the dimensions to add a channel dimension for grayscale images
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Convert labels to one-hot encoded format
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Performing Maxpooling

In [ ]:
def LeNetwithMaxpooling():
    cnn_model = models.Sequential()
    cnn_model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
    cnn_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    cnn_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(layers.Flatten())
    cnn_model.add(layers.Dense(32, activation='relu'))
    cnn_model.add(layers.Dense(10, activation='softmax'))
    return cnn_model

def Training(cnn_model, training_data, validation_data, model_name, num_epochs=3, batch_size=32):
    cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    train_history = cnn_model.fit(
        training_data[0], training_data[1],
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=validation_data
    )
    test_loss, test_accuracy = cnn_model.evaluate(validation_data[0], validation_data[1], verbose=2)
    print(f"{model_name} Test Accuracy: {test_accuracy:.4f}")
    return train_history, test_loss, test_accuracy

# Building and training the max pooling model
optimized_cnn_model = LeNetwithMaxpooling()
print("Training LeNet model using Max Pooling layer:")
history, test_loss, test_accuracy = Training(
    optimized_cnn_model,
    training_data=(x_train, y_train),
    validation_data=(x_test, y_test),
    model_name="Max Pooled"
)


Training LeNet model using Max Pooling layer:
Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 45s 24ms/step - accuracy: 0.8811 - loss: 0.3977 - val_accuracy: 0.9827 - val_loss: 0.0523
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 84s 25ms/step - accuracy: 0.9803 - loss: 0.0632 - val_accuracy: 0.9861 - val_loss: 0.0403
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 89s 28ms/step - accuracy: 0.9867 - loss: 0.0434 - val_accuracy: 0.9869 - val_loss: 0.0392
313/313 - 2s - 6ms/step - accuracy: 0.9869 - loss: 0.0392
Max Pooled Test Accuracy: 0.9869


# Performing Average Pooling

In [ ]:
def LeNetwithAvgPooling():
    cnn_model = models.Sequential()
    cnn_model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
    cnn_model.add(layers.AveragePooling2D(pool_size=(2, 2)))  # Average Pooling Layer
    cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    cnn_model.add(layers.AveragePooling2D(pool_size=(2, 2)))  # Average Pooling Layer
    cnn_model.add(layers.Flatten())
    cnn_model.add(layers.Dense(32, activation='relu'))
    cnn_model.add(layers.Dense(10, activation='softmax'))
    return cnn_model

# Building and training the average pooling model
optimized_avg_pooling_model = LeNetwithAvgPooling()
print("\nTraining LeNet model using Average Pooling layer:")

avg_pooling_history, avg_pooling_test_loss, avg_pooling_test_accuracy = Training(
    optimized_avg_pooling_model,
    training_data=(x_train, y_train),
    validation_data=(x_test, y_test),
    model_name="Average Pooled"
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training LeNet model using Average Pooling Layer:
Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.8357 - loss: 0.5409 - val_accuracy: 0.9707 - val_loss: 0.0906
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 37s 20ms/step - accuracy: 0.9710 - loss: 0.0948 - val_accuracy: 0.9799 - val_loss: 0.0623
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 46s 22ms/step - accuracy: 0.9810 - loss: 0.0639 - val_accuracy: 0.9810 - val_loss: 0.0566
313/313 - 2s - 8ms/step - accuracy: 0.9810 - loss: 0.0566
Average Pooled Test Accuracy: 0.9810


# Performing Fuzzy Pooling

In [ ]:
class FuzzyPoolingLayer(layers.Layer):
    def __init__(self, pool_size=(2, 2)):
        super(FuzzyPoolingLayer, self).__init__()
        self.pool_size = pool_size
    # the three triangular membership functions
    def first_membership(self, x, d, c):
        return tf.where(x < c, 1.0, tf.where(x <= d, (d - x) / (d - c), 0.0))

    def second_membership(self, x, a, m, b):
        return tf.where(x < a, 0.0, tf.where(x <= m, (x - a) / (m - a), tf.where(x < b, (b - x) / (b - m), 0.0)))

    def third_membership(self, x, r, q):
        return tf.where(x < r, 0.0, tf.where(x <= q, (x - r) / (q - r), 1.0))

    def call(self, inputs):
        # parameters for the membership functions
        pool_height, pool_width = self.pool_size
        d, c = 3.0, 1.0
        a, m, b = 1.5, 3.0, 4.5
        r, q = 3.0, 4.5

        # Patches from CNN
        patches = tf.image.extract_patches(
            images=inputs,
            sizes=[1, pool_height, pool_width, 1],
            strides=[1, pool_height, pool_width, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )
        # Reshape
        batch_size, out_height, out_width, num_channels = tf.shape(inputs)[0], tf.shape(patches)[1], tf.shape(patches)[2], inputs.shape[-1]
        patch_depth = pool_height * pool_width
        patches = tf.reshape(patches, (-1, patch_depth, num_channels))

        # Calculate membership for each patch
        pi_1 = self.first_membership(patches, d, c)
        pi_2 = self.second_membership(patches, a, m, b)
        pi_3 = self.third_membership(patches, r, q)

        # Aggregate fuzzy scores [sum of membership values in each layer]
        spi_1 = tf.reduce_sum(pi_1, axis=1)
        spi_2 = tf.reduce_sum(pi_2, axis=1)
        spi_3 = tf.reduce_sum(pi_3, axis=1)

        # Arg max step
        scores = tf.stack([spi_1, spi_2, spi_3], axis=1)
        max_score_idx = tf.argmax(scores, axis=1)
        max_score_idx = tf.reshape(max_score_idx, (-1, 1, num_channels))

        #Create the final Fuzzy patch by scores
        pi_dash = tf.where(
            tf.equal(max_score_idx, 0),
            pi_1,
            tf.where(
                tf.equal(max_score_idx, 1),
                pi_2,
                pi_3
            )
        )

        # Defuzzification using Center of Gravity(COG)
        numerator = tf.reduce_sum(pi_dash * patches, axis=1)
        denominator = tf.reduce_sum(pi_dash, axis=1)
        returnlayer = tf.where(denominator != 0, numerator / denominator, tf.zeros_like(numerator))

        return tf.reshape(returnlayer, (batch_size, out_height, out_width, num_channels))


In [ ]:

def build_fuzzy_pooling_cnn():
    cnn_model = models.Sequential()
    cnn_model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
    cnn_model.add(FuzzyPoolingLayer(pool_size=(2, 2)))  # Custom fuzzy pooling layer
    cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    cnn_model.add(layers.Flatten())
    cnn_model.add(layers.Dense(32, activation='relu'))
    cnn_model.add(layers.Dense(10, activation='softmax'))
    return cnn_model

# Building and training the fuzzy pooling model
optimized_fuzzy_model = build_fuzzy_pooling_cnn()
print("\nTraining LeNet using Fuzzy Pooling layer:")

fuzzy_pooling_history, fuzzy_pooling_test_loss, fuzzy_pooling_test_accuracy = Training(
    optimized_fuzzy_model,
    training_data=(x_train, y_train),
    validation_data=(x_test, y_test),
    model_name="Fuzzy Pooling"
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training LeNet using Fuzzy Pooling layer:
Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 109s 57ms/step - accuracy: 0.8863 - loss: 0.3665 - val_accuracy: 0.9795 - val_loss: 0.0620
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 139s 55ms/step - accuracy: 0.9811 - loss: 0.0650 - val_accuracy: 0.9857 - val_loss: 0.0443
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 101s 54ms/step - accuracy: 0.9874 - loss: 0.0401 - val_accuracy: 0.9880 - val_loss: 0.0380
313/313 - 6s - 19ms/step - accuracy: 0.9880 - loss: 0.0380
Fuzzy Pooling Test Accuracy: 0.9880
